In [1]:
import os
import time
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing
from bs4 import BeautifulSoup

import chromedriver_binary
from selenium import webdriver

In [2]:
def download(link, no_of_movies):

    file_name = link.lower()
    
    link = 'https://www.imdb.com/search/title/?genres=' + link
    
    res = requests.get(link)
    soup = BeautifulSoup(res.content, 'html.parser')
    data = []

    page = 0

    try:
        for _ in tqdm(range(int(no_of_movies/50))):

            votes = np.nan
            gross = np.nan

            for m in soup.find('div', class_ = 'lister-list').find_all('div', class_ = 'lister-item mode-advanced'):

                try:
                    id_         = m.find('div', class_ = 'lister-item-image float-left').find('a').get('href').split('/')[-2].strip()
                except:
                    id_         = np.nan

                try:
                    name        = m.find('h3').find('a').text
                except:
                    name        = np.nan

                try:
                    year        =  m.find('h3').find('span', class_ = 'lister-item-year').text
                except:
                    year        = np.nan

                try:
                    certificate = m.find('p', class_ = "text-muted").find('span', class_ = 'certificate').text.strip()
                except:
                    certificate = np.nan

                try:
                    duration    = m.find('p', class_ = "text-muted").find('span', class_ = 'runtime').text.strip()
                except:
                    duration    = np.nan

                try:    
                    genre       = m.find('p', class_ = "text-muted").find('span', class_ = 'genre').text.strip()
                except:
                    genre       = np.nan

                try:
                    rating      = m.find('div', class_ = 'ratings-bar').find('strong').text.strip()
                except:
                    rating      = np.nan

                try:
                    description = m.find_all('p', class_ = 'text-muted')[1].text.strip()
                except:
                    description = np.nan



                try:     # If were having both directors and stars
                    temp = BeautifulSoup(str(m.find_all('p')[2]).split('<span class="ghost">|</span>')[0], 'html.parser')
                    if ('Director' in temp.text):

                        directors_id   = ','.join([i.get('href').split('/')[-2] for i in temp.find_all('a')])
                        directors_name = ','.join([i.text.strip() for i in temp.find_all('a')])

                    temp = BeautifulSoup(str(m.find_all('p')[2]).split('<span class="ghost">|</span> ')[1], 'html.parser')
                    if ("Star" in temp.text):

                        stars_id   = ','.join([i.get('href').split('/')[-2] for i in temp.find_all('a')])
                        stars_name = ','.join([i.text.strip() for i in temp.find_all('a')])
                except:

                    directors_id   = np.nan
                    stars_id       = np.nan
                    directors_name = np.nan
                    stars_name     = np.nan


                try:       # Directors but not Stars
                    if('Director' in m.find_all('p')[2].text and 'Stars' not in m.find_all('p')[2].text):
                        directors_id   = ','.join([i.get('href') for i in m.find_all('p')[2].find_all('a')])
                        directors_name = ','.join([i.text.strip() for i in m.find_all('p')[2].find_all('a')])
                except:
                    directors_id = np.nan
                    directors_name = np.nan

                try:        # Stars but not Directors
                    if('Stars' in m.find_all('p')[2].text and 'Director' not in m.find_all('p')[2].text):
                        stars_id     = ','.join([i.get('href').split('/')[-2] for i in m.find_all('p')[2].find_all('a')])
                        stars_name   = ','.join([i.text.strip() for i in m.find_all('p')[2].find_all('a')])
                except:
                    stars_id = np.nan
                    stars_name = np.nan


                try:
                    if len(m.find('p', class_ = 'sort-num_votes-visible').find_all('span')) == 2:

                        if('Vote' in m.find('p', class_ = 'sort-num_votes-visible').find_all('span')[0].text):    
                            votes =  m.find('p', class_ = 'sort-num_votes-visible').find_all('span')[1].text.strip()

                        elif('Gros' in m.find('p', class_ = 'sort-num_votes-visible').find_all('span')[0].text):    
                            gross =  m.find('p', class_ = 'sort-num_votes-visible').find_all('span')[1].text.strip()

                    else:
                        votes = m.find('p', class_ = 'sort-num_votes-visible').find_all('span')[1].get('data-value')
                        gross = m.find('p', class_ = 'sort-num_votes-visible').find_all('span')[4].get('data-value')
                except:

                    votes = np.nan
                    gross = np.nan

                data.append([id_, name, year, rating ,certificate, duration, genre, votes , gross ,directors_id, 
                             directors_name , stars_id, stars_name, description])



            next_page_link = 'https://www.imdb.com' + soup.find('a',class_ = 'lister-page-next next-page').get('href')

            res = requests.get(next_page_link)
            soup = BeautifulSoup(res.content, 'html.parser')

    except:
        pass
    
    df = pd.DataFrame(data, columns = ['id', 'name', 'year', 'rating' ,'certificate', 'duration', 'genre', 'votes'
                               , 'gross_income' ,'directors_id', 'directors_name' , 'stars_id', 'stars_name', 
                               'description'])
    
    df.to_csv(file_name + '.csv', index = False)

In [21]:
download( 'Film-Noir', 834)

100%|███████████████████████████████████████████| 16/16 [00:32<00:00,  2.02s/it]


In [22]:
download ('war'          , 52693 )
download ('crime'        , 402897)  
download ('drama'        , 2580167)
download ('comedy'       , 1936679)
download ('talk-show'    , 1318249 )
download ('short'        , 1117705 )
download ('romance'      , 938806 )
download ('documentary'  , 883693 )
download ('news'         , 883562 )
download ('family'       , 767551 )
download ('reality-tv'   , 511942 )
download ('music'        , 497372 )
download ('animation'    , 467363 )
download ('crime'        , 402897 )
download ('action'       , 359609 )
download ('adventure'    , 354780 )
download ('game-show'    , 337213 )
download ('fantasy'      , 317814 )
download ('thriller'     , 293344 )
download ('sport'        , 257641 )
download ('mystery'      , 250481 )
download ('sci-fi'       , 214400 )
download ('horror'       , 190218 )
download ('history'      , 165526 )
download ('musical'      , 119298 )
download ('biography'    , 99933 )
download ('western'      , 36097 )
download ('film-noir'    , 834 )
download ('adult'        , 52 )

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (3904947575.py, line 31)